In [1]:
import findspark
findspark.init()

import pyspark
findspark.find()

'C:\\spark-3.3.1-bin-hadoop2\\spark-3.3.1-bin-hadoop2'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql import Row

In [2]:
try:
    sc.stop()
except:
    print("SparkContext has not been initiated")

SparkContext has not been initiated


In [3]:
conf = (SparkConf() 
    .set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.postgresql:postgresql:42.5.0")
    .setMaster("local") #spark://spark-master:7077
    .setAppName('SparkApp') 
)

sc = SparkContext(conf=conf)

spark = SparkSession(sc)

spark

# Load dataframe

In [4]:
player_df = spark.read \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option('uri', "mongodb://localhost:27017/football_data_new.player") \
    .load()
    
# player_df.printSchema()
# player_df.head(n=5)

# Get schema

In [5]:
info_table_schema = player_df.select('info').schema[0].dataType

In [6]:
stats_table_names = player_df.schema['stats'].dataType.fieldNames()

stats_table_schemas = {}
for table_name in stats_table_names:
    full_table_name = 'stats.' + table_name
    table_schema = player_df.select(full_table_name).schema[0].dataType.elementType
    stats_table_schemas[table_name] = table_schema

# Process table

In [11]:
from tqdm import tqdm

In [15]:
stats_tables = {}
info_table = spark.createDataFrame(
    data = [],
    schema = info_table_schema
)

for player_row in tqdm(player_df.collect()):
    player_id = player_row['_id']
    player_info = player_row['info']
    player_stats = player_row['stats']
    
    # Parse Info table
    try:
        info_df = spark.createDataFrame([player_info], schema=info_table_schema) 
        info_table = info_table.union(info_df)
    except Exception as e: 
        print(e)
        print("ERROR when parsing info", player_id, player_info)
    
    # Parse Stats tables
    for table_name in stats_table_names:
        table_data = player_stats[table_name]
        if not table_data:
            continue

        table_schema = stats_table_schemas[table_name]

        try:
            sub_stats_df = spark.createDataFrame(table_data, schema=table_schema) 
            sub_stats_df = sub_stats_df.withColumn("PlayerID", lit(player_id))
            sub_stats_df = sub_stats_df.withColumn("PlayerName", lit(player_info['ShortName']))

            if table_name in stats_tables:
                stats_tables[table_name] = stats_tables[table_name].union(sub_stats_df)
            else:
                stats_tables[table_name] = sub_stats_df    
        except Exception as e: 
            print(e)
            print("ERROR when parsing stats", player_id, table_name)

100%|██████████| 733/733 [15:57<00:00,  1.31s/it]


## Process Info

In [16]:
info_table.show(n=5)

+--------------+--------------------+--------------------+------+------+------+----------+-----------+--------+
|     ShortName|            FullName|            Position|Height|Weight|Footed|       DOB|Nationality|    Club|
+--------------+--------------------+--------------------+------+------+------+----------+-----------+--------+
|  Nicolas Pépé|                null| FW-MF (AM, right...| 178cm|  68kg|  Left|1995-05-29|        CIV|132ebc33|
| Nathan Fraser|       Nathan Fraser|                  FW|  null|  null|  null|2005-02-22|        IRL|a8fbbb12|
| Nikola Vlašić|                null|      FW-MF (AM) ▪  | 178cm|  76kg| Right|1997-10-04|        CRO|105360fe|
|Arnaut Danjuma|Arnaut Danjuma Gr...|           FW-MF ▪  | 187cm|  73kg| Right|1997-01-31|        NED|361ca564|
|  Jimmy Morgan|    Jimmy Jay Morgan|                  FW|  null|  null|  null|2006-01-21|        ENG|179c33c7|
+--------------+--------------------+--------------------+------+------+------+----------+-----------+--

In [22]:
import re
from pyspark.sql.types import *

def proc_str(input, func):
    if isinstance(input, str):
        return func(input)
    else:
        return input

def proc_Height(input):
    if not isinstance(input, str):
        return input
    return int(re.findall("\d+", input)[0])

proc_Height_udf = udf(proc_Height, IntegerType())

def proc_Position(input):
    if not isinstance(input, str):
        return input
    return input.replace('▪', '').strip()

proc_Position_udf = udf(proc_Position, StringType())

In [25]:
# info_table.withColumn('Height_new', proc_Height_udf(info_table.Height))
info_table.show(n=5)

+--------------+--------------------+--------------------+------+------+------+----------+-----------+--------+
|     ShortName|            FullName|            Position|Height|Weight|Footed|       DOB|Nationality|    Club|
+--------------+--------------------+--------------------+------+------+------+----------+-----------+--------+
|  Nicolas Pépé|                null| FW-MF (AM, right...| 178cm|  68kg|  Left|1995-05-29|        CIV|132ebc33|
| Nathan Fraser|       Nathan Fraser|                  FW|  null|  null|  null|2005-02-22|        IRL|a8fbbb12|
| Nikola Vlašić|                null|      FW-MF (AM) ▪  | 178cm|  76kg| Right|1997-10-04|        CRO|105360fe|
|Arnaut Danjuma|Arnaut Danjuma Gr...|           FW-MF ▪  | 187cm|  73kg| Right|1997-01-31|        NED|361ca564|
|  Jimmy Morgan|    Jimmy Jay Morgan|                  FW|  null|  null|  null|2006-01-21|        ENG|179c33c7|
+--------------+--------------------+--------------------+------+------+------+----------+-----------+--

In [26]:
info_table.toPandas().to_csv('output/player_info.csv')

In [16]:
info_table.collect()[0]

Py4JJavaError: An error occurred while calling o197321.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.compu

In [ ]:
def process_text(text):
    if not isinstance(text, str):
        return text

info_table.withColumn("Upper_Name", )

## Process Stats

In [30]:
for table_name in stats_tables:
    table_data = stats_tables[table_name]
    print(table_name)
    table_data.show(n=5)
    print()

defensive


Py4JJavaError: An error occurred while calling o197154.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 18) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrR

In [28]:
string_1 = 'FW-MF (AM, right) ▪  '
string_1 = string_1.replace('▪', '').strip()
string_1

'FW-MF (AM, right)'

# Save table

In [ ]:
(info_table.write.format("jdbc")
    .option("url", "jdbc:postgresql://localhost:5432/news_crawled")
    .option("driver", "org.postgresql.Driver")
    .option("dbtable", "player_info")
    .option("user", "test1")
    .option("password", "test1")
    .save()
)

In [18]:
for table_name in stats_tables:
    full_table_name = 'player_' + table_name
    table_df = stats_tables[table_name]
    
    (table_df.write.format("jdbc")
        .option("url", "jdbc:postgresql://localhost:5432/news_crawled")
        .option("driver", "org.postgresql.Driver")
        .option("dbtable", full_table_name)
        .option("user", "test1")
        .option("password", "test1")
        .save()
    )
    

Py4JJavaError: An error occurred while calling o197341.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 14) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe